In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import requests
import pandas as pd

%cd /content/drive/My Drive/GitHub/minor-goodbooks-recommender

/content/drive/My Drive/GitHub/minor-goodbooks-recommender


In [6]:
# Sample URL to fetch the html page
url = "https://www.goodreads.com/review/list/88851923-yuriy?print=true"
  
# Headers to mimic the browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
}
  
# Get the page through get() method
html_response = requests.get(url=url, headers = headers)

In [7]:
html_response

<Response [200]>

In [2]:
goodreads_id = '88851923'

In [3]:
  url = f"https://www.goodreads.com/review/list/{goodreads_id}?print=true"
  headers = {
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
      (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }

  page = requests.get(url=url, headers = headers).text

  for i in range(2, 11):
    url = f"https://www.goodreads.com/review/list/{goodreads_id}?page={i}&print=true"
    page += requests.get(url=url, headers = headers).text

  from bs4 import BeautifulSoup
  soup = BeautifulSoup(page, 'html.parser')

NameError: ignored

In [ ]:
page

str

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_response.text, 'html.parser')

In [ ]:
type(soup)

bs4.BeautifulSoup

In [20]:
import re

In [24]:
html_response.text

'<html>\n<head>\n  <title>Yuriy’s books on Goodreads (92 books)</title>\n\n<meta content=\'Yuriy has 92 books on his all shelf: A Single Man by Christopher Isherwood, Lock &amp; West by Alexander C. Eberhart, The War at Troy by Lindsay Clarke, The ...\' name=\'description\'>\n<meta content=\'telephone=no\' name=\'format-detection\'>\n<link href=\'https://www.goodreads.com/review/list/88851923\' rel=\'canonical\'>\n  <meta property="og:title" content="Yuriy’s books on Goodreads (92 books)"/>\n  <meta property="og:type" content="website"/>\n  <meta property="og:site_name" content="Goodreads"/>\n  <meta property="og:description" content="Yuriy has 92 books on his all shelf: A Single Man by Christopher Isherwood, Lock &amp; West by Alexander C. Eberhart, The War at Troy by Lindsay Clarke, The ..."/>\n    <meta property="og:image" content="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1380688373l/16842.jpg"/>\n  <meta property="og:url" content="https://www.goodreads.

In [29]:
re.findall('books on Goodreads \((.*) books\)', html_response.text)

['92', '92', '92']

In [22]:
type(soup.find_all('div'))

bs4.element.ResultSet

In [ ]:
[title.get('class') for title in soup.find_all('div')]

In [ ]:
titles = []
for title in soup.find_all('a'):
    titles.append(title.get('title'))

In [ ]:
titles

In [ ]:
titles = [title.get('title') for title in soup.find_all('a')]

In [ ]:
bad_stuff = ['Goodreads Home', None, 'My group discussions', 'Messages', 'Friends', 'did not like it', 'it was ok', 'liked it', 'really liked it', 'it was amazing',]

In [ ]:
books = [i for i in titles if i not in bad_stuff]

In [ ]:
len(books)

92

In [ ]:
ratings = []
for rating in soup.find_all('span'):
    ratings.append(rating.get('class'))

In [ ]:
rates = [i.get('class') for i in soup.find_all('span')]

In [ ]:
rates

In [ ]:
bad_stuff_ratings = [None,
                     ['headerPersonalNav__icon', 'headerPersonalNav__icon--notifications'],
                     ['headerPersonalNav__icon', 'headerPersonalNav__icon--discussions'],
                     ['headerPersonalNav__icon', 'headerPersonalNav__icon--inbox'],
                     ['headerPersonalNav__icon', 'headerPersonalNav__icon--friendRequests'],
                     ['controlGroup', 'uitext'],
                     ['bookMeta'],
                     ['greyText'],
                     ['greyText', 'smallText'],
                     ['greyText'],
                     ['', 'staticStars', 'notranslate'],
                     ['darkGreyText'], 
                     ['date_started_value'], 
                     ['date_read_value'],
                     ['previous_page', 'disabled'],
                     ['next_page', 'disabled'],]

In [ ]:
stars = [i[1] for i in ratings if i not in bad_stuff_ratings]

In [ ]:
stars

In [ ]:
#from itertools import izip

def groupwise(iterable):
    a = iter(iterable)
    return zip(a, a, a, a, a)

rates = []
for s1, s2, s3, s4, s5 in groupwise(stars):
   rates.append((s1, s2, s3, s4, s5).count('p10'))

In [ ]:
len(rates)

92



---



In [40]:
books = pd.read_csv('data/books.csv')
book_map = books[['id', 'title', 'authors']]

In [ ]:
goodreads_id = '88851914-adam'

In [ ]:
"https://www.goodreads.com/review/list/88851923-yuriy?print=true"

In [47]:
def fetch_user_ratings_goodreads(goodreads_id):
  import requests
  url = f"https://www.goodreads.com/review/list/{goodreads_id}?print=true"
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
  (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
  html_output = requests.get(url=url, headers = headers).text

  import re
  num_books = int(re.findall('books on Goodreads \((.*) books\)', html_output)[0])

  for p in range(2, (num_books // 20 + (num_books % 20 > 0) + 1)):
    url = f"https://www.goodreads.com/review/list/{goodreads_id}?page={p}&print=true"
    html_output += requests.get(url=url, headers = headers).text

  from bs4 import BeautifulSoup
  soup = BeautifulSoup(html_output, 'html.parser')

  bad_stuff_titles = ['Goodreads Home', None, 'My group discussions', 'Messages', 'Friends', 
                      'did not like it', 'it was ok', 'liked it', 'really liked it', 'it was amazing',]
  books = [i.get('title') for i in soup.find_all('a') if i.get('title') not in bad_stuff_titles]
  
  bad_stuff_ratings = [None,
                      ['headerPersonalNav__icon', 'headerPersonalNav__icon--notifications'],
                      ['headerPersonalNav__icon', 'headerPersonalNav__icon--discussions'],
                      ['headerPersonalNav__icon', 'headerPersonalNav__icon--inbox'],
                      ['headerPersonalNav__icon', 'headerPersonalNav__icon--friendRequests'],
                      ['controlGroup', 'uitext'],
                      ['bookMeta'],
                      ['greyText'],
                      ['greyText', 'smallText'],
                      ['greyText'],
                      ['', 'staticStars', 'notranslate'],
                      ['darkGreyText'], 
                      ['date_started_value'], 
                      ['date_read_value'],
                      ['previous_page', 'disabled'],
                      ['next_page', 'disabled'],]
  stars = [i.get('class')[1] for i in soup.find_all('span') if i.get('class') not in bad_stuff_ratings]

  def groupwise(iterable):
    a = iter(iterable)
    return zip(a, a, a, a, a)

  ratings = [(s1, s2, s3, s4, s5).count('p10') for (s1, s2, s3, s4, s5) in groupwise(stars)]

  df = pd.merge(pd.DataFrame({'title' : books, 'rating' : ratings}), book_map, on='title')
  df = df.drop('title', axis=1).rename({'id' : 'book_id'}, axis=1)
  df['user_id'] = [999999] * len(df)
  df = df.reindex(columns=['user_id', 'book_id', 'rating'])
  df = df.loc[df['rating'] != 0]

  if df.empty:
    raise ValueError('No matching books rated')
  else: return df

In [49]:
fetch_user_ratings_goodreads('88851914')

,user_id,book_id,rating
0,999999,8390,5
1,999999,7606,5
2,999999,474,1
3,999999,6272,5
4,999999,3930,5
5,999999,772,5
6,999999,161,5
7,999999,155,5
8,999999,19,5


In [42]:
user_ratings

NameError: ignored

In [ ]:
books = pd.read_csv('books.csv')

In [ ]:
book_map = books[['id', 'title']]

In [ ]:
fetch_user_ratings_goodreads('88851923')

,user_id,book_id,rating
1,999999,5,2
15,999999,576,3
22,999999,14,1
23,999999,155,1
24,999999,19,3
25,999999,54,1
27,999999,255,5
28,999999,8,2
29,999999,6055,4
30,999999,357,5


In [ ]:
pd.merge(fetch_user_ratings_goodreads('88851923'), book_map.rename({'id' : 'book_id'}, axis = 1), on='book_id')

,user_id,book_id,rating,title
0,999999,5,2,The Great Gatsby
1,999999,576,3,Candide
2,999999,14,1,Animal Farm
3,999999,155,1,"The Two Towers (The Lord of the Rings, #2)"
4,999999,19,3,The Fellowship of the Ring (The Lord of the Ri...
5,999999,54,1,The Hitchhiker's Guide to the Galaxy (Hitchhik...
6,999999,255,5,Atlas Shrugged
7,999999,8,2,The Catcher in the Rye
8,999999,6055,4,Three Comrades
9,999999,357,5,All Quiet on the Western Front


In [50]:
def fancy_title(title):
  return '<b>' + title + '</b>' + '\n' + str(book_map.loc[book_map['title']==title, 'authors'].values[0])

In [51]:
fancy_title('The Hobbit')

'<b>The Hobbit</b>\nJ.R.R. Tolkien'